## Spanish-English Translator using a Sequence-to-Sequence Model with Encoder-Decoder Transformers

This notebook is based on the example in the book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). Chp-11 to run in Google Colab.

**It was modified in certain features and adapted to run in the following environment:**

- Windows 11
- Python 3.10.9
- TensorFlow 2.10.1
- VSCode 1.75.1
- CUDA 11.2
- CUDNN 11.2
- GPU: NVIDIA GeoForce RTX 2060
- CPU: 06 Intel Core i7-10750 @2.6Ghz

### A Spanish-English machine translation 

The data is downloaded from http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip and unziped in local disk.

In [3]:
text_file = "spa-eng/spa.txt"
with open(text_file, encoding='utf-8') as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [23]:
import random
for _ in range(10):
    print(random.choice(text_pairs))

('Tom writes poetry.', '[start] Tom escribe poesía. [end]')
('I know one thing we could do that might be fun.', '[start] Sé una cosa que podríamos hacer que puede ser divertida. [end]')
('Take care.', '[start] Cuidaos. [end]')
('Tom died in combat.', '[start] Tom murió en combate. [end]')
('She almost fainted.', '[start] Ella casi se desmayó. [end]')
('The new method was anything but ideal.', '[start] El nuevo método era todo menos ideal. [end]')
("Tom didn't have enough time to do anything else.", '[start] Tom no tenía tiempo suficiente para hacer nada más. [end]')
('Tom listens.', '[start] Tom escucha. [end]')
('Is the tap water in Australia drinkable?', '[start] ¿Es potable el agua del grifo en Australia? [end]')
('She was the first one to help him.', '[start] Ella fue la primera en ayudarle. [end]')


In [5]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**Vectorizing the English and Spanish text pairs**

In [7]:
import tensorflow as tf
import string
import re
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

**Preparing datasets for the translation task**

In [8]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [9]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


### Sequence-to-sequence learning with RNNs

**GRU-based encoder**

In [10]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

**GRU-based decoder and the end-to-end model**

In [11]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**Training our recurrent sequence-to-sequence model**

In [12]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 138s 98ms/step - loss: 1.6345 - accuracy: 0.4159 - val_loss: 1.3303 - val_accuracy: 0.5021
Epoch 2/15
1302/1302 [==============================] - 124s 95ms/step - loss: 1.3183 - accuracy: 0.5258 - val_loss: 1.1671 - val_accuracy: 0.5664
Epoch 3/15
1302/1302 [==============================] - 122s 94ms/step - loss: 1.1755 - accuracy: 0.5754 - val_loss: 1.0867 - val_accuracy: 0.5988
Epoch 4/15
1302/1302 [==============================] - 122s 94ms/step - loss: 1.0840 - accuracy: 0.6071 - val_loss: 1.0504 - val_accuracy: 0.6159
Epoch 5/15
1302/1302 [==============================] - 122s 94ms/step - loss: 1.0357 - accuracy: 0.6309 - val_loss: 1.0369 - val_accuracy: 0.6258
Epoch 6/15
1302/1302 [==============================] - 122s 94ms/step - loss: 1.0076 - accuracy: 0.6491 - val_loss: 1.0316 - val_accuracy: 0.6313
Epoch 7/15
1302/1302 [==============================] - 122s 94ms/step - loss: 0.9872 - accuracy: 0.6635 - val_loss: 1

**Translating new sentences with our RNN encoder and decoder**

In [13]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
How lucky I am to meet you here!
1/1 [==============================] - 0s 26ms/step
[start] qué tan a menudo me [UNK] aquí [end]
-
He ran outside naked.
1/1 [==============================] - 0s 29ms/step
[start] Él se quedó sin decir [end]
-
Above the sink are several whisky bottles.
1/1 [==============================] - 0s 27ms/step
[start] por favor los [UNK] son [UNK] [end]
-
I'm a little disappointed with my boys.
1/1 [==============================] - 0s 50ms/step
[start] estoy un poco de mis hijos con los [UNK] [end]
-
That is the house where he lives.
1/1 [==============================] - 0s 46ms/step
[start] esa es la casa en la que vive [end]
-
The dog house is outside.
1/1 [==============================] - 0s 45ms/step
[start] el perro de casa está fuera [end]
-
Tom said he was terrified of snakes.
1/1 [==============================] - 0s 46ms/step
[start] tom dijo que él estaba las de acuerdo [end]
-
Only peace can save the world.
1/1 [==============================]

### Sequence-to-sequence learning with Transformer

#### The Transformer Encoder
**`Transformer Encoder` implemented as a subclassed `Layer`**

In [17]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import LayerNormalization

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        # self.attention = layers.MultiHeadAttention(
        self.attention = MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

#### The Transformer decoder

**The `TransformerDecoder`**

In [18]:
# from tensorflow.keras.layers import MultiHeadAttention
# from tensorflow.keras.layers import LayerNormalization

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: A Transformer for machine translation

**PositionalEmbedding layer**

In [19]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [20]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**Training the sequence-to-sequence Transformer**

In [21]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 98s 73ms/step - loss: 1.6932 - accuracy: 0.4095 - val_loss: 1.3543 - val_accuracy: 0.4991
Epoch 2/30
1302/1302 [==============================] - 95s 73ms/step - loss: 1.3465 - accuracy: 0.5285 - val_loss: 1.1837 - val_accuracy: 0.5675
Epoch 3/30
1302/1302 [==============================] - 96s 73ms/step - loss: 1.1917 - accuracy: 0.5799 - val_loss: 1.1040 - val_accuracy: 0.5990
Epoch 4/30
1302/1302 [==============================] - 97s 74ms/step - loss: 1.0997 - accuracy: 0.6129 - val_loss: 1.0590 - val_accuracy: 0.6194
Epoch 5/30
1302/1302 [==============================] - 96s 74ms/step - loss: 1.0498 - accuracy: 0.6353 - val_loss: 1.0257 - val_accuracy: 0.6331
Epoch 6/30
1302/1302 [==============================] - 97s 75ms/step - loss: 1.0188 - accuracy: 0.6516 - val_loss: 1.0133 - val_accuracy: 0.6416
Epoch 7/30
1302/1302 [==============================] - 98s 76ms/step - loss: 0.9969 - accuracy: 0.6639 - val_loss: 1.0113 -

**Translating new sentences with our Transformer model**

In [22]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Tom was out all day.
[start] tom estaba fuera todo el día [end]
-
I don't want to drink cold tea.
[start] no quiero tomar un vaso de té [end]
-
You desperately need a vacation.
[start] [UNK] en que necesitas unas vacaciones [end]
-
I know the story from beginning to end.
[start] conozco a la historia de final [end]
-
Wait one second.
[start] espera un segundo [end]
-
I want to help you, but I can't.
[start] quiero ayudarte con años pero no [end]
-
What do you do for a living?
[start] qué haces un trabajo [end]
-
Do you know who made it?
[start] sabes quién es eso [end]
-
The room was hot.
[start] el cuarto estaba caliente [end]
-
Can't you separate fantasy and reality from each other?
[start] no sabes que la [UNK] y no [UNK] los demás [end]
-
He's an Englishman.
[start] Él es un inglés [end]
-
The blue lines on the map represent rivers.
[start] las [UNK] azul en el mapa de la [UNK] [end]
-
Let's encourage the players.
[start] [UNK] la madre de los a los que [UNK] [end]
-
He seems to 